<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Inspect-data" data-toc-modified-id="Inspect-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Inspect data</a></span></li><li><span><a href="#Save-for-later-use" data-toc-modified-id="Save-for-later-use-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save for later use</a></span></li><li><span><a href="#Excess-Market-return" data-toc-modified-id="Excess-Market-return-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Excess Market return</a></span></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data cleaning</a></span></li><li><span><a href="#Variables-calculations" data-toc-modified-id="Variables-calculations-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Variables calculations</a></span></li><li><span><a href="#Descriptive-statistics-tabel" data-toc-modified-id="Descriptive-statistics-tabel-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Descriptive statistics tabel</a></span></li></ul></div>

In [178]:
%load_ext autoreload
%autoreload 2

"""
Load relevant packages 
"""

import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import matplotlib.ticker as mtick
from matplotlib import animation, cm
from scipy import stats
import pandas_datareader as pdr
import datetime
import seaborn as sns
import matplotlib.ticker as ticker
import datetime
from scipy.stats import chi2
import pickle
from scipy import optimize
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import kurtosis
from scipy.stats import skew

from typing import Union, List

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data

In [179]:
"""
Import risk free rate
"""
df_rf = pd.DataFrame(pd.read_excel("/Users/julielaustsen/Desktop/OneDrive/UNI/Speciale /Data/Risk free rate/euriobor-ready_to_python.xlsx", index_col=0, header=1,sheet_name='to python') )
df_rf = df_rf.drop(columns=['Unnamed: 2'])
df_rf.head()

,euribor 3-month (Y)
Date,
1999-01-24,0.031321
1999-02-24,0.030934
1999-03-24,0.030467
1999-04-24,0.026965
1999-05-24,0.025790


In [180]:
"""
Import market data
"""

df_mkt = pd.DataFrame(pd.read_excel("/Users/julielaustsen/Desktop/OneDrive/UNI/Speciale /Data/Market data/market data to python.xlsx", index_col=0, header=1,sheet_name='to python') )
df_mkt.head()

,EUROPE IMI (price),EUROPE IMI (return)
Date,,
1999-01-24,702.872,0.000000
1999-02-24,710.217,0.010450
1999-03-24,727.125,0.023807
1999-04-24,764.111,0.050866
1999-05-24,742.658,-0.028076


In [397]:
"""
Import bank data 
"""
path = "/Users/julielaustsen/Desktop/OneDrive/UNI/Speciale /Data/Eikon"
os.chdir(path) 

#importér data    
df = pd.read_excel('Eikon_data_to_python.xlsx', index_col=0)

In [398]:
"""
Create dictionary with the banks as keys
"""
#list with banks' name
no_banks = np.int32(df.shape[1]/15) #Antal banker
banks_names = [] #empty array
for i in np.arange(no_banks):
    banks_names.append(df.columns[i*15].split(' - MARKET',1)[0]) 
    
#dictionary with all banks
data = {}
for index, value in enumerate(banks_names): #enumerate gives the index and thee value
    data[value] = df.iloc[:,index*15:index*15+15]

#export a bank to excel - to check
#data['SPAREBANKEN VEST'].to_excel('sparebanken vest.xlsx')

### Inspect data

In [383]:
"""
Control numbers of banks and keys
Note there is a difference between the numbers of banks and keys in the dictionary
"""
print(len(banks_names))
print(len(data.keys()))
print(no_banks)

234
234
234


In [384]:
"""
Count of unique bank names. It list the banks that are mentioned more than 1 time
The dictionary will always take the last observation in the dataset. 
BANCA ANTONVENETA DEAD - 06/04/06 appeared 7 times by mistake. This is changed in excel data now. 
"""
compa = {i:banks_names.count(i) for i in banks_names}
for i in compa:
    if compa[i] > 1:
        print(i)

In [311]:
"""
Save data on disk
"""
#gem dicionary på disk
with open('data', 'wb') as data_dict:
   pickle.dump(data, data_dict)

#save np.array on disk so we don't have to import all data again
with open('data_matrix', 'wb') as data_numpy:
   pickle.dump(data_matrix, data_numpy)    
    

#load data
with open('data', 'rb') as data_dict:
    data = pickle.load(data_dict)

with open('data_matrix', 'rb') as data_numpy:
    data_matrix = pickle.load(data_numpy)



### Save for later use

In [386]:
"""
Save for later comparing to lists
"""
main_list = list(set(banks_names) - set(data.keys()))
main_list

[]

In [ ]:
"""
Dictionary to np-array - FRGR save for later
"""
#Create empty numpy array (3D) for all the data
dim_1 = data[list(data.keys())[0]].shape[0]
dim_2 = data[list(data.keys())[0]].shape[1]
dim_3 = len(banks_names)

data_matrix = np.empty([dim_1,dim_2,dim_3])

#Load data from dictionary to np.array
for index, value in enumerate(banks_names):
    data_matrix[:,:,index] = data[value]


______________________

### Excess Market return

In [184]:
"""
Calculating excess return
"""
excess_mkt= df_mkt['EUROPE IMI (return)'] - df_rf['euribor 3-month (Y)']
df_mkt.insert(2, 'EUROPE IM (excess)', excess_mkt) #Adding the excess market return to the existing dataframe

In [215]:
"""
Creating a empty np.array to fill our calculations into
"""
dim_1 = data[list(data.keys())[0]].shape[0]
dim_2 = data[list(data.keys())[0]].shape[1]
dim_3 = len(banks_names)

calc_data = np.empty([dim_1-1,11,dim_3]) #11 is the number of variables we want

### Data cleaning

In [400]:
"""
We want the same amount of observations for market capitalization and price, so when we are calculating returns
we are able to do value weighted portfolios. We therefore create a new dictionary where a market value and price
both got the value nan if one of the variable is missing. 
"""
data_nan = {} #create a new empty dictionary
for i in data.keys():
    data_nan[i] = data[i]
    data_nan[i].loc[data_nan[i].iloc[:,:2].isnull().any(axis=1), :] = np.nan #we replace with the value nan, if a observation is missing mkt cap or price

In [401]:
"""
Now we want to remove any banks that might have all values = nan. We create a new dictionary where these are removed
"""
data_nan_re = data_nan.copy() 
for i in data_nan.keys():
    if np.int32(data_nan_re[i].isna().sum().mean()) == data_nan_re[i].shape[0]: #remove the bank/key if the mean of nan is = shape of the dataframe
        del data_nan_re[i]

In [402]:
"""
We see that we have removed 8 banks/keys
"""
print(len(data_nan_re.keys()))
print(len(data_nan.keys()))

226
234


### Variables calculations

In [491]:
"""
Dimension of our cleaned data
"""
dim_1 = data_nan_re[list(data_nan_re.keys())[0]].shape[0]
dim_2 = data_nan_re[list(data_nan_re.keys())[0]].shape[1]
dim_3 = len(data_nan_re.keys())

In [463]:
"""
Creating a dictionary with the variables we need for calculations
"""
data_calc = {}
for i in data_nan_re.keys():
    data_calc[i] = pd.DataFrame(data_nan_re[i].iloc[:,0])
    ret = data_nan_re[i].iloc[:,1].diff()/data_nan_re[i].iloc[1:,0]
    data_calc[i]['Return'] = ret
    data_calc[i]['Excess Return'] = ret - df_rf['euribor 3-month (Y)'][1:]
    equ = data_nan_re[i].iloc[:,7] - data_nan_re[i].iloc[:,9]
    data_calc[i]['Book Equity'] = equ
    data_calc[i]['Book Leverage ratio'] = data_nan_re[i].iloc[:,9] / data_nan_re[i].iloc[:,6]
    data_calc[i]['Book Equity ratio'] = equ / data_nan_re[i].iloc[:,6]
    data_calc[i]['Book Equity/RWA'] = equ / data_nan_re[i].iloc[:,10]
    data_calc[i]['Tier 1 ratio'] = data_nan_re[i].iloc[:,13] / data_nan_re[i].iloc[:,6]
    data_calc[i]['Tier 1/RWA'] = data_nan_re[i].iloc[:,13] / data_nan_re[i].iloc[:,10]
    data_calc[i]['Total Assets'] = data_nan_re[i].iloc[:,6]
    data_calc[i]['Total Risk Weighted Assets'] = data_nan_re[i].iloc[:,10]

In [471]:
#How to see data with numbers
data_calc[list(data_calc.keys())[0]].head()

,HSBC HOLDINGS - MARKET VALUE,Return,Excess Return,Book Equity,Book Leverage ratio,Book Equity ratio,Book Equity/RWA,Tier 1 ratio,Tier 1/RWA,Total Assets,Total Risk Weighte Assets
Date,,,,,,,,,,,
1999-01-24,21058.787791,NaN,NaN,22938283.0,0.934854,0.065146,0.110247,0.050161,0.084888,352108246.0,208061994.0
1999-02-24,22972.391653,0.001694,-0.029240,22938283.0,0.934854,0.065146,0.110247,0.050161,0.084888,352108246.0,208061994.0
1999-03-24,24795.379785,0.001113,-0.029354,22938283.0,0.934854,0.065146,0.110247,0.050161,0.084888,352108246.0,208061994.0
1999-04-24,31032.689088,0.004034,-0.022931,22938283.0,0.934854,0.065146,0.110247,0.050161,0.084888,352108246.0,208061994.0
1999-05-24,29293.251345,-0.002291,-0.028081,22938283.0,0.934854,0.065146,0.110247,0.050161,0.084888,352108246.0,208061994.0


### Descriptive statistics tabel

In [508]:
"""
Create a empty dataframe and fill it with descriptive statistics
"""
descriptive1=pd.DataFrame(columns=['Observation','Mean', 'Median', 'Min', 'Max'])

#returns
descriptive1.loc['Return','Observation']= '{:,.0f}'.format(dim_1*dim_3-np.count_nonzero(np.isnan(np.array([data_calc[i].iloc[:,1] for i in data_calc.keys()]))),4)
descriptive1.loc['Return', 'Mean']= round(np.nanmean(np.array([data_calc[i].iloc[:,1] for i in data_calc.keys()])),4)#nanmean=Compute the arithmetic mean along the specified axis, ignoring NaNs.
descriptive1.loc['Return', 'Median']= round(np.nanmedian(np.array([data_calc[i].iloc[:,1] for i in data_calc.keys()])),4) #nanmedian=Compute the median along the specified axis, while ignoring NaNs.
descriptive1.loc['Return', 'Min']= round(np.nanmin(np.array([data_calc[i].iloc[:,1] for i in data_calc.keys()])),4)
descriptive1.loc['Return', 'Max']= round(np.nanmax(np.array([data_calc[i].iloc[:,1] for i in data_calc.keys()])),4)

#Market capitalization
descriptive1.loc['Market capitalization in mill','Observation']= '{:,.0f}'.format(dim_1*dim_3-np.count_nonzero(np.isnan(np.array([data_calc[i].iloc[:,0] for i in data_calc.keys()]))),4)
descriptive1.loc['Market capitalization in mill', 'Mean']= '{:,.0f}'.format(np.nanmean(np.array([data_calc[i].iloc[:,0] for i in data_calc.keys()])),0)
descriptive1.loc['Market capitalization in mill', 'Median']= '{:,.0f}'.format(np.nanmedian(np.array([data_calc[i].iloc[:,0] for i in data_calc.keys()])),0)
descriptive1.loc['Market capitalization in mill', 'Min']= '{:,.0f}'.format(np.nanmin(np.array([data_calc[i].iloc[:,0] for i in data_calc.keys()])),0)
descriptive1.loc['Market capitalization in mill', 'Max']= '{:,.0f}'.format(np.nanmax(np.array([data_calc[i].iloc[:,0] for i in data_calc.keys()])),0)

#Market excess return
descriptive1.loc['Market excess return','Observation'] = 275-df_mkt['EUROPE IM (excess)'].isna().sum()
descriptive1.loc['Market excess return', 'Mean']= round(df_mkt['EUROPE IM (excess)'].mean(skipna=True),4)
descriptive1.loc['Market excess return', 'Median']= round(df_mkt['EUROPE IM (excess)'].median(skipna=True),4)
descriptive1.loc['Market excess return', 'Min']= round(df_mkt['EUROPE IM (excess)'].min(skipna=True),4)
descriptive1.loc['Market excess return', 'Max']= round(df_mkt['EUROPE IM (excess)'].max(skipna=True),4)

#risk free rate
descriptive1.loc['Risk-free rate','Observation'] = 276
descriptive1.loc['Risk-free rate', 'Mean']= round(df_rf['euribor 3-month (Y)'].mean(skipna=True),4)
descriptive1.loc['Risk-free rate', 'Median']= round(df_rf['euribor 3-month (Y)'].median(skipna=True),4)
descriptive1.loc['Risk-free rate', 'Min']= round(df_rf['euribor 3-month (Y)'].min(skipna=True),4)
descriptive1.loc['Risk-free rate', 'Max']= round(df_rf['euribor 3-month (Y)'].max(skipna=True),4)

#Book leverage ratio
descriptive1.loc['Book leverage ratio','Observation']= '{:,.0f}'.format(dim_1*dim_3-np.count_nonzero(np.isnan(np.array([data_calc[i].iloc[:,4] for i in data_calc.keys()]))),4)
descriptive1.loc['Book leverage ratio', 'Mean']= round(np.nanmean(np.array([data_calc[i].iloc[:,4] for i in data_calc.keys()])),4)
descriptive1.loc['Book leverage ratio', 'Median']= round(np.nanmedian(np.array([data_calc[i].iloc[:,4] for i in data_calc.keys()])),4) 
descriptive1.loc['Book leverage ratio', 'Min']= round(np.nanmin(np.array([data_calc[i].iloc[:,4] for i in data_calc.keys()])),4)
descriptive1.loc['Book leverage ratio', 'Max']= round(np.nanmax(np.array([data_calc[i].iloc[:,4] for i in data_calc.keys()])),4)

#Book equity ratio
descriptive1.loc['Book equity ratio','Observation']= '{:,.0f}'.format(dim_1*dim_3-np.count_nonzero(np.isnan(np.array([data_calc[i].iloc[:,5] for i in data_calc.keys()]))),4)
descriptive1.loc['Book equity ratio', 'Mean']= round(np.nanmean(np.array([data_calc[i].iloc[:,5] for i in data_calc.keys()])),4)
descriptive1.loc['Book equity ratio', 'Median']= round(np.nanmedian(np.array([data_calc[i].iloc[:,5] for i in data_calc.keys()])),4) 
descriptive1.loc['Book equity ratio', 'Min']= round(np.nanmin(np.array([data_calc[i].iloc[:,5] for i in data_calc.keys()])),4)
descriptive1.loc['Book equity ratio', 'Max']= round(np.nanmax(np.array([data_calc[i].iloc[:,5] for i in data_calc.keys()])),4)

#Tier 1 ratio
descriptive1.loc['Tier 1 ratio','Observation']= '{:,.0f}'.format(dim_1*dim_3-np.count_nonzero(np.isnan(np.array([data_calc[i].iloc[:,7] for i in data_calc.keys()]))),4)
descriptive1.loc['Tier 1 ratio', 'Mean']= round(np.nanmean(np.array([data_calc[i].iloc[:,7] for i in data_calc.keys()])),4)
descriptive1.loc['Tier 1 ratio', 'Median']= round(np.nanmedian(np.array([data_calc[i].iloc[:,7] for i in data_calc.keys()])),4) 
descriptive1.loc['Tier 1 ratio', 'Min']= round(np.nanmin(np.array([data_calc[i].iloc[:,7] for i in data_calc.keys()])),4)
descriptive1.loc['Tier 1 ratio', 'Max']= round(np.nanmax(np.array([data_calc[i].iloc[:,7] for i in data_calc.keys()])),4)

In [510]:
"""
Comment to Book leverage ratio max value at 39.9383 and book equity ratio min value due to the Portuguese 
bank Banco Espírito Santo. The bank was split into two (healthy and toxi) operations and it caused the 
equity value to become negative in  January 2014 which offsets in the liabilities was higher than total 
liabilities and equity. The bank was delisted in February 2016. https://en.wikipedia.org/wiki/Banco_Esp%C3%ADrito_Santo 

We should probably do the outlier analysis before the tabel below.

Market value - wired numbers here. We need to check up on the definition in refinitive whether it is 
given in mill. We might consindering calculating market capitalization our self and use that instead however this is 
again complicated to calculated when consindering everything has to be calculated in euros.
Difference between market value and market capitalization https://finance.zacks.com/market-value-vs-market-capitalization-3644.html
However it makes senes the bank is Österreichische Volksbanken. The market value of 0.04 is from December 2015
just before the bank is liquidated. 
"""


descriptive1

,Observation,Mean,Median,Min,Max
Return,"41,583",-0.0039,0.0,-70.25,8.2684
Market capitalization in mill,"41,809","7,485",793,0,"176,091"
Market excess return,275,-0.0125,-0.0063,-0.1861,0.1449
Risk-free rate,276,0.0155,0.0107,-0.0058,0.0511
Book leverage ratio,"40,632",0.9271,0.9298,0.0084,34.191
Book equity ratio,"40,632",0.0729,0.0702,-33.191,0.9916
Tier 1 ratio,"33,930",0.0681,0.0601,-0.0011,0.6406


In [142]:
"""
To control the calculations in excel
"""
pd.DataFrame(calc_data[:,0,:]).to_excel('test.xlsx')